In [23]:
import os
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all firecrawl-py PyMuPDF

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-0.2.21-py3-none-any.whl.metadata (6.0 kB)
Using cached langchain_core-0.2.21-py3-none-any.whl (372 kB)


In [24]:
local_llm = 'gemma'
# local_llm = 'llama3'

In [25]:
## index

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import FireCrawlLoader, PyMuPDFLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document

urls = [
    "https://www.ai-jason.com/learning-ai/how-to-reduce-llm-cost",
    # "https://www.ai-jason.com/learning-ai/gpt5-llm",
    # "https://www.ai-jason.com/learning-ai/how-to-build-ai-agent-tutorial-3"
]

# Load documents from URLs
doc = []
for url in urls:
    loader = FireCrawlLoader(api_key="fc-75d116f86f7d4eceb112c6c44d624508", url=url, mode='scrape')
    try:
        loaded_docs = loader.load()
        doc.append(loaded_docs)
    except Exception as e:
        print(f"Error loading {url}: {e}")

# Flatten the list of lists
docs_list = [item for sublist in doc for item in sublist]

# Load PDF documents
pdf_file_paths = ["data/Medical Policy FY 23-24.pdf", "data/Provident fund policy.pdf"]

for pdf_path in pdf_file_paths:
    loader = PyMuPDFLoader(pdf_path)
    try:
        loaded_docs = loader.load()
        docs_list.extend(loaded_docs)
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=250, chunk_overlap=0)
doc_splitter = text_splitter.split_documents(docs_list)

# Filter and clean metadata
filtered_doc = []
for doc in doc_splitter:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))


# Load text file content
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return Document(page_content=content, metadata={"source": file_path})


# Path to your text file
text_file_path = "useful_questions.txt"
text_doc = load_text_file(text_file_path)

# Split the text file content
text_doc_splitter = text_splitter.split_documents([text_doc])

# Filter and clean metadata for the text file content
filtered_text_doc = []
for doc in text_doc_splitter:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
        filtered_text_doc.append(Document(page_content=doc.page_content, metadata=clean_metadata))

# Combine URL documents, PDF documents, and text file documents
combined_docs = filtered_doc + filtered_text_doc

# Add to vectorDB
embedding = GPT4AllEmbeddings(model_name="all-MiniLM-L6-v2.gguf2.f16.gguf", gpt4all_kwargs={'allow_download': 'True'})
vectorstore = Chroma.from_documents(
    documents=combined_docs,
    collection_name="rag-chroma",
    embedding=embedding,
)

retriever = vectorstore.as_retriever()


Simple Rag Chain v1.0

In [26]:
### Generate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

# Prompt
prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Use the following context to answer the question. If you dont understand the answer, ask another question.
    Question: {question} 
    Context: {context} 
    Answer: assistant""",
    input_variables=["question", "context"],
)


llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "What are the key differences between the direct reimbursement model and the insurance claim model for health coverage?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)
print("---------------------------------------------------------")
print("\n\n")

**Key Differences between Direct Reimbursement Model and Insurance Claim Model:**

**Direct Reimbursement Model:**

* Employees pay for medical expenses out-of-pocket.
* They submit receipts and other documentation to HR for reimbursement.
* Reimbursement is made in any amount at any time up to the annual limit.


**Insurance Claim Model:**

* Health coverage is provided through an insurance company.
* Employees present their health insurance card at the time of treatment.
* The insurance company directly pays for covered expenses.
---------------------------------------------------------





In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model for semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define prompt templates
context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Use the following context to answer the question. Avoid phrases like "Based on the provided context" and explain the answer in the end.
    Question: {question}
    Context: {context}
    Answer: assistant""",
    input_variables=["question", "context"],
)

no_context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Answer the question concisely and directly.
    Question: {question}
    Answer: assistant""",
    input_variables=["question"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Enhanced Function to check relevance using semantic similarity
def is_relevant(question, context_docs, threshold=0.3):
    context_text = format_docs(context_docs)

    # Split context into smaller chunks
    context_chunks = context_text.split('\n\n')

    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)

    max_similarity = 0
    for chunk in context_chunks:
        # Encode each chunk of context
        context_embedding = model.encode(chunk, convert_to_tensor=True)

        # Compute similarity
        similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
        max_similarity = max(max_similarity, similarity.item())

    # Return True if similarity is above threshold  
    return max_similarity > threshold

# Function to get the appropriate chain based on relevance
def get_chain(question, context_docs):
    if is_relevant(question, context_docs):
        print("Relevant")
        return context_prompt | llm | StrOutputParser()
    else:
        print("Irrelevant")
        return no_context_prompt | llm | StrOutputParser()

# Main function to run the question-answering process
def answer_question(question):
    docs = retriever.invoke(question)
    rag_chain = get_chain(question, docs)
    if is_relevant(question, docs):
        generation = rag_chain.invoke({"context": docs, "question": question})
    else:
        generation = rag_chain.invoke({"question": question})
    return generation

# Example question
question = "What are the key differences between the direct reimbursement model and the insurance claim model for health coverage?"
answer = answer_question(question)
print(answer)


Relevant
**Key Differences between Direct Reimbursement Model and Insurance Claim Model:**

**Direct Reimbursement Model:**

* Employees directly pay for medical expenses.
* Reimbursement is provided by the company up to an annual limit of PKR 100,000.
* Employees need to submit receipts and other documentation to HR for reimbursement.

**Insurance Claim Model:**

* Coverage is provided through an insurance company.
* Employees visit hospitals or labs from a list provided and present their health insurance card.
* The insurance company directly pays for covered expenses.


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# Load a pre-trained model for semantic similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define prompt templates
context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Use the following context to answer the question. Avoid phrases like "Based on the provided context" and explain the answer in the end.
    Question: {question}
    Context: {context}
    Answer: assistant""",
    input_variables=["question", "context"],
)

no_context_prompt = PromptTemplate(
    template="""system You are an assistant for question-answering tasks.
    Answer the question concisely and directly.
    Question: {question}
    Answer: assistant""",
    input_variables=["question"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Load a text classification model for future usefulness prediction
usefulness_classifier = pipeline("text-classification", model="bert-base-uncased")

# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Enhanced Function to check relevance using semantic similarity
def is_relevant(question, context_docs, threshold=0.3):
    context_text = format_docs(context_docs)

    # Split context into smaller chunks
    context_chunks = context_text.split('\n\n')

    # Encode the question
    question_embedding = model.encode(question, convert_to_tensor=True)

    max_similarity = 0
    for chunk in context_chunks:
        # Encode each chunk of context
        context_embedding = model.encode(chunk, convert_to_tensor=True)

        # Compute similarity
        similarity = util.pytorch_cos_sim(question_embedding, context_embedding)
        max_similarity = max(max_similarity, similarity.item())

    # Return True if similarity is above threshold  
    return max_similarity > threshold

# Function to determine if a question/statement is useful for the future
def is_useful_for_future(question):
    # Use the classification pipeline to predict usefulness
    result = usefulness_classifier(question)
    is_useful = result[0]['label'] == 'USEFUL'
    if is_useful:
        with open("useful_question.txt", "a") as file:
            file.write(question + "\n")


    return result[0]['label'] == 'IMPORTANT'

# Function to get the appropriate chain based on relevance
def get_chain(question, context_docs):
    if is_relevant(question, context_docs):
        print("Relevant")
        return context_prompt | llm | StrOutputParser()
    else:
        print("Irrelevant")
        return no_context_prompt | llm | StrOutputParser()

# Main function to run the question-answering process
def answer_question(question):
    docs = retriever.invoke(question)
    rag_chain = get_chain(question, docs)
    if is_relevant(question, docs):
        generation = rag_chain.invoke({"context": docs, "question": question})
    else:
        generation = rag_chain.invoke({"question": question})

    # Check if the question is useful for the future
    if is_useful_for_future(question):
        print(f"This question/statement is useful for the future.")
    else:
        print(f"This question/statement is not particularly useful for the future.")

    return generation

# Example question
question = "What is the deadline for submitting expense claims for direct reimbursement?"
answer = answer_question(question)
print(answer)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Relevant
This question/statement is not particularly useful for the future.
**The deadline for submitting expense claims for direct reimbursement is the 23rd of any month.**

The provided text states that "Expense claims will be received up to 23rd of any month for reimbursement with salary."
